# ODA data by Sector (with donor and recipient)
This tutorial shows you how to build a Pandas DataFrame containing ODA data from any number of donors to any number of recipients, across sectors available in the list of indicators, over a range of years.

The tutorial has the following steps:
1. Import `ODAData`, the main tool used to interact with the data
2. Set the arguments
3. Create an instance of `ODAData` with the specific arguments you would like to use
4. Load the indicators and get the DataFrame
5. Optionally, export the DataFrame

## 1. Import ODAData

We can gather the data we need using the `ODAData` class. An object from this class can:
- Get data for specific indicators
- Optionally, filter the data for specific donors, recipients and years
- Optionally, exchange and deflate data

`oda_data` automatically downloads the relevant datasets (i.e. DAC1, DAC2a or CRS tables) depending on the indicators selected in step 3.
It is highly recommended that you specify the folder where you want to store, and from where you want to read, this raw data. This must be done once in each notebook or script where we use the `oda_data` package. You can specify the data path if the raw data has already been downloaded, or if you haven't yet specified the download path in your notebook and script.

In [2]:
from oda_data import ODAData, set_data_path

# in section 3, we give the option to retreive data for specific donor and recipient groupings, so need to import the following
from oda_data.tools.groupings import donor_groupings, recipient_groupings

# If you haven't set the data path, you can do it now.
set_data_path(path="../tutorials/data")

# 2. Set the arguments for the DataFrame

Next, we need to set the right arguments, which are used to create the right DataFrame.
- *years*: you must specify the `years`, as an `int`, `list` or `range`
- *donors*: you can _optionally_ specify the `donors` you want the output to have (as `int`, or `list`) of donor codes.
- *recipients*: you can _optionally_ specify the `recipients` you want the output to have. Not all indicators
    need or accept recipients. If using an indicator for which recipients are not an option, a warning will be logged to
    the console and the recipients ignored for that indicator.
- *currency*: you can _optionally_ specify the `currency` in which you want your data to be shown. If not specified,
    by default, `USD` will be used. Other options include `EUR`, `GBP` and `CAN`.
- *prices*: you can _optionally_ specify the `prices` in which you want your data to be shown. If not specified,
    by default, `current` will be used. The other option is `constant`. If specifying `constant` a `base_year` must be set.
- *base_year*: you must specify a `base_year` if you have set `prices = 'constant'`. If you have chosen `current` prices,
    by default, `base_year` will be `None`.


Below are some example settings for this tutorial. For clarity, we will first store them in variables, but you can
always pass them directly as arguments to the `ODAData` class.

We can retrieve the data for specific donor and recipient groupings. First, assign recipient_groupings() to the variable groupings. Then print the grouping keys to see the available options (the equivalent works for donors).

In [3]:
groupings = recipient_groupings()

In [5]:
print(groupings.keys())

dict_keys(['all_recipients', 'all_developing_countries_regions', 'african_countries', 'africa_regional', 'african_countries_regional', 'sahel', 'ldc_countries', 'france_priority', 'dac2a_aggregates'])


In [11]:
# Select LCDs
ldc_countries = groupings['ldc_countries']

In [13]:
# Select years as (for example) a range. Remember ranges are exclusive of the upper bound.
years = range(2018,2021)

# Select donors, which must be specified by their codes. To get all donors, do not use this argument.
donors = [4, 5, 12, 302]

# Select recipients, which must be specified by their codes or available country groupings. In this example, we are selecting LDCs (which we define below).
recipients = ldc_countries

# Select the currency. By default 'USD' is shown but we'll get the data in Euros.
currency = 'EUR'

# Select the prices. By default, 'current' is shown, but we'll get the data in constant prices.
prices = 'constant'

# Set the base year. We must set this given that we've asked for constant data.
base_year = 2021

In [ ]:
ODAData().available_donors()

In [ ]:
ODAData().available_recipients()

In [ ]:
ODAData().available_currencies()

Finally, we can specify the indicators we want to retrieve for our analysis.
    A full list of indicators can be seen by using the `.available_indicators()` method.

In [ ]:
ODAData().available_indicators()

For this tutorial, we will use the 'crs_gender_significant_flow_disbursement_gross', 'crs_gender_principal_flow_disbursement_gross', and 'crs_gender_total_flow_gross' indicators.

In [9]:
indicators = ['crs_gender_significant_flow_disbursement_gross',
              'crs_gender_principal_flow_disbursement_gross',
              'crs_gender_total_flow_gross']

## 3. Create an instance of ODAData

In order to get the data we want, we will create an instance of the `ODAData` class, using the arguments we specified above.

We will store this instance in a variable called `oda`, which we will use later to load the indicators and get the DataFrame we're after.


In [15]:
oda = ODAData(years=years,
              recipients=list(recipients),
              donors=donors,
              currency=currency,
              prices=prices,
              base_year=base_year,
              include_names=True)

## 4. Load the indicators and get the DataFrame

Then we loop through all your assigned indicators and tell ODAData to load them. ODAData will now download the relevant datasets, so this may take some time.

In [16]:
# Load the indicators
oda.load_indicator(indicators)

# Get a DataFrame with all the data. Here, we also add the share of total.
df = oda.add_share_of_total(True).get_data()

# Finally, group the DataFrame rows by year, currency, prices and indicator
df = df.groupby(['donor_name','year','currency','prices','indicator'], observed=True, dropna=False)[['donor_name','indicator','value','share']].sum(numeric_only=True)

## 5. Optionally export DataFrame as CSV

Finally, we can export the DataFrame as a CSV if required.

In [17]:
df.to_csv(r'../tutorials/output/total_gender_oda.csv', index = False)